# Project 1 - Oregon Wildfire Data

In [ ]:
import json
import requests
from config import api_key
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np

In [ ]:
weather_data_path = "portland_weather_data.csv"
weather_data = pd.read_csv(weather_data_path)

weather_data

In [ ]:
weather_data_2 = weather_data.copy()

weather_data_2['DATE'] = pd.to_datetime(weather_data_2['DATE'])

weather_data_2

In [ ]:
monthly_summary = weather_data_2.groupby(pd.Grouper(key="DATE", freq="M")).mean()
monthly_summary = monthly_summary.reset_index()
monthly_summary

In [ ]:
plt.plot(monthly_summary["DATE"], monthly_summary["TAVG"])
plt.show()